In [28]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob

In [29]:
def get_epochs_path(pace):
    # Define the relative path pattern for .edf files in all subdirectories of the Data directory
    path_pattern = os.path.join('..', '..', 'CleanedEpochs', 'Recording', 'Subj*', f'*{pace}*')
    # Use glob to get all matching files
    epoch_files = glob(path_pattern, recursive=True)
    return epoch_files

fast_df = [mne.read_epochs(epoch) for epoch in get_epochs_path('Fast')]
slow_df = [mne.read_epochs(epoch) for epoch in get_epochs_path('Slow')]

Reading e:\Campus\RM\FinalProject\Codes\Analysis & Classification\..\..\CleanedEpochs\Recording\Subj1\Fast-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    1992.19 ms
        0 CTF compensation matrices available
Not setting metadata
555 matching events found
No baseline correction applied
0 projection items activated
Reading e:\Campus\RM\FinalProject\Codes\Analysis & Classification\..\..\CleanedEpochs\Recording\Subj10\Fast-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    1992.19 ms
        0 CTF compensation matrices available
Not setting metadata
464 matching events found
No baseline correction applied
0 projection items activated
Reading e:\Campus\RM\FinalProject\Codes\Analysis & Classification\..\..\CleanedEpochs\Recording\Subj11\Fast-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    1992.19 ms
        0 CTF compensation matrices available
Not setting metadata
465 matching events found
No baseline correction 

In [30]:
len(fast_df), len(slow_df)

(16, 16)

In [31]:
fast_epochs = mne.concatenate_epochs(fast_df)
slow_epochs = mne.concatenate_epochs(slow_df)

Not setting metadata
7050 matching events found
No baseline correction applied
Not setting metadata
7900 matching events found
No baseline correction applied


In [34]:
fast_group = np.concatenate([[i] * len(fast_df[i]) for i in range(len(fast_df))])
slow_group = np.concatenate([[i] * len(slow_df[i]) for i in range(len(slow_df))])

fast_label = np.concatenate([[1]*len(fast_df[i]) for i in range(len(fast_df))])
slow_label = np.concatenate([[0]*len(slow_df[i]) for i in range(len(slow_df))])

In [36]:
# Combine the fast and slow epochs
data = mne.concatenate_epochs([fast_epochs, slow_epochs])
group = np.concatenate([fast_group, slow_group])
label = np.concatenate([fast_label, slow_label])
print(len(data), len(group), len(label))

Not setting metadata
14950 matching events found
No baseline correction applied
14950 14950 14950


In [37]:
from mne.time_frequency import psd_array_welch
def eeg_power_band(epochs):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5 * n_channels]
        Transformed data.
    """
    # specific frequency bands
    FREQ_BANDS = {
        "delta": [1, 4],
        "theta": [4, 8],
        "alpha": [8, 12],
        "beta": [12, 30],
        "gamma": [30, 50],
    }
    
    sfreq = epochs.info['sfreq']  # Sampling frequency
    epoch_length = epochs.times[-1] - epochs.times[0]  # Duration of each epoch in seconds

    # Set `n_per_seg` to match the duration of each epoch
    n_per_seg = int(sfreq * epoch_length)

    # Set `n_fft` to match `n_per_seg` for computational efficiency
    n_fft = n_per_seg

    # Compute PSD using Welch's method
    psds, freqs = psd_array_welch(epochs.get_data(copy=False), sfreq=sfreq, n_fft=n_fft, n_per_seg=n_per_seg, n_overlap=n_per_seg // 2)
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

In [38]:
features = []
for d in range(len(data)):
    features.append(eeg_power_band(data[d]))

Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective wind

In [39]:
features_cat = np.concatenate(features, axis=0)
print(features_cat.shape)

(14950, 70)


In [47]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold

In [50]:
def classify(model):
    clf = make_pipeline(StandardScaler(), model)
    gkf = GroupKFold(n_splits=5)
    scores = cross_val_score(clf, features_cat, label, cv=gkf, groups=group)
    return scores

In [54]:
# Logistic Regression
model = LogisticRegression()
scores = classify(model)

print(f'Logistic Regression: {scores.mean().round(2)} (+/- {scores.std().round(2)})')

Logistic Regression: 0.65 (+/- 0.08)


In [55]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100)
scores = classify(model)

print(f'Random Forest: {scores.mean().round(2)} (+/- {scores.std().round(2)})')

Random Forest: 0.63 (+/- 0.1)


In [56]:
# XGBoost
from xgboost import XGBClassifier

model = XGBClassifier(use_label_encoder=False)
scores = classify(model)

print(f'XGBoost: {scores.mean().round(2)} (+/- {scores.std().round(2)})')

XGBoost: 0.62 (+/- 0.08)


In [57]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
scores = classify(model)

print(f'KNN: {scores.mean().round(2)} (+/- {scores.std().round(2)})')

KNN: 0.61 (+/- 0.07)


# Deep Learning

In [70]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(features_cat.shape[1],)))
# model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 128)               9088      
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dense_14 (Dense)            (None, 32)                2080      
                                                                 
 dense_15 (Dense)            (None, 1)                 33        
                                                                 
Total params: 19,457
Trainable params: 19,457
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GroupKFold

def create_model(input_shape):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(input_shape,)))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def classify_nn(features, labels, groups, epochs=50, batch_size=32):
    scores = []
    gkf = GroupKFold(n_splits=5)

    for train_index, test_index in gkf.split(features, labels, groups):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        model = create_model(features.shape[1])

        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                  validation_split=0.2, callbacks=[], verbose=1, shuffle=True)
        
        _, acc = model.evaluate(X_test, y_test, verbose=0)
        scores.append(acc)
        
        tf.keras.backend.clear_session()  # Clear session to free resources
        
    return scores

# Example usage
scores = classify_nn(features_cat, label, group)
print(f"Mean CV Score = {np.mean(scores):.3f}, Std = {np.std(scores):.3f}")


Epoch 1/50
1161/1161 [==============================] - 11s 9ms/step - loss: 0.5784 - accuracy: 0.6787 - val_loss: 1.3055 - val_accuracy: 0.2838
Epoch 2/50
1161/1161 [==============================] - 9s 8ms/step - loss: 0.4903 - accuracy: 0.7472 - val_loss: 1.4169 - val_accuracy: 0.3708
Epoch 3/50
1161/1161 [==============================] - 9s 8ms/step - loss: 0.4618 - accuracy: 0.7654 - val_loss: 1.0699 - val_accuracy: 0.5624
Epoch 4/50
1161/1161 [==============================] - 9s 8ms/step - loss: 0.4408 - accuracy: 0.7791 - val_loss: 1.3014 - val_accuracy: 0.5250
Epoch 5/50
1161/1161 [==============================] - 9s 8ms/step - loss: 0.4321 - accuracy: 0.7923 - val_loss: 1.8165 - val_accuracy: 0.2898
Epoch 6/50
1161/1161 [==============================] - 9s 8ms/step - loss: 0.4219 - accuracy: 0.7994 - val_loss: 1.4279 - val_accuracy: 0.4531
Epoch 7/50
1161/1161 [==============================] - 10s 8ms/step - loss: 0.4076 - accuracy: 0.8090 - val_loss: 1.4850 - val_accurac

KeyboardInterrupt: 